In [0]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
gdrive/  hvc_annotations.csv  resized/  sample_data/


In [0]:
!ls

In [0]:
%tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.applications.resnet50 import ResNet50

from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [0]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

,gender,imagequality,age,weight,carryingbag,footwear,emotion,bodypose,image_path
0,male,Average,35-45,normal-healthy,Grocery/Home/Plastic Bag,Normal,Neutral,Front-Frontish,resized/1.jpg
1,female,Average,35-45,over-weight,None,Normal,Angry/Serious,Front-Frontish,resized/2.jpg
2,male,Good,45-55,normal-healthy,Grocery/Home/Plastic Bag,CantSee,Neutral,Front-Frontish,resized/3.jpg
3,male,Good,45-55,normal-healthy,Daily/Office/Work Bag,Normal,Neutral,Front-Frontish,resized/4.jpg
4,female,Good,35-45,slightly-overweight,None,CantSee,Neutral,Front-Frontish,resized/5.jpg


In [0]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

,0,1,2,3,4
image_path,resized/1.jpg,resized/2.jpg,resized/3.jpg,resized/4.jpg,resized/5.jpg
gender_female,0,1,0,0,1
gender_male,1,0,1,1,0
imagequality_Average,1,1,0,0,0
imagequality_Bad,0,0,0,0,0
imagequality_Good,0,0,1,1,1
age_15-25,0,0,0,0,0
age_25-35,0,0,0,0,0
age_35-45,1,1,0,0,1
age_45-55,0,0,1,1,0


In [0]:
import keras
import numpy as np

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=32, shuffle=True,augmentation=None):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augmentation=augmentation

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        if self.augmentation is not None:
          image = self.augmentation.flow(image,shuffle=False).next()
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


In [0]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.15)
train_df.shape, val_df.shape

((11537, 28), (2036, 28))

In [0]:
train_df.head()

,image_path,gender_female,gender_male,imagequality_Average,imagequality_Bad,imagequality_Good,age_15-25,age_25-35,age_35-45,age_45-55,age_55+,weight_normal-healthy,weight_over-weight,weight_slightly-overweight,weight_underweight,carryingbag_Daily/Office/Work Bag,carryingbag_Grocery/Home/Plastic Bag,carryingbag_None,footwear_CantSee,footwear_Fancy,footwear_Normal,emotion_Angry/Serious,emotion_Happy,emotion_Neutral,emotion_Sad,bodypose_Back,bodypose_Front-Frontish,bodypose_Side
9138,resized/9139.jpg,0,1,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0
5842,resized/5843.jpg,0,1,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,0,1,0
5019,resized/5020.jpg,1,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0
1857,resized/1858.jpg,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,1,0,0,1,0
3390,resized/3391.jpg,1,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,0,1,0


In [0]:
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=32,augmentation=ImageDataGenerator(horizontal_flip=True,shear_range=0.2,width_shift_range=0.1,height_shift_range=0.1))
valid_gen = PersonDataGenerator(val_df, batch_size=64, shuffle=False)

In [0]:
# get number of output units from data
images, targets = next(iter(train_gen))
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

{'age': 5,
 'bag': 3,
 'emotion': 4,
 'footwear': 3,
 'gender': 2,
 'image_quality': 3,
 'pose': 3,
 'weight': 4}

In [0]:
from keras.layers import Conv2D,BatchNormalization,MaxPooling2D

In [0]:
# backbone = VGG16(
#     weights=None,
#     include_top=False, 
#     input_tensor=Input(shape=(224, 224, 3))
# )
input_img = Input(shape=(224, 224, 3))

model = Conv2D(16, (3, 3), padding='same', activation='relu')(input_img)
model =BatchNormalization()(model)
model = Dropout(0.1)(model)

model = Conv2D(32, (3, 3), padding='same', activation='relu')(model)
model =BatchNormalization()(model)
model = Dropout(0.1)(model)

model = Conv2D(10, (1, 1), padding='same', activation='relu')(model)
model = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(model)

model = Conv2D(16, (3, 3), padding='same', activation='relu')(model)
model =BatchNormalization()(model)
model = Dropout(0.1)(model)

model = Conv2D(16, (3, 3), padding='same', activation='relu')(model)
model =BatchNormalization()(model)
model = Dropout(0.1)(model)

model = Conv2D(16, (3, 3), padding='same', activation='relu')(model)
model =BatchNormalization()(model)
model = Dropout(0.1)(model)

model = Conv2D(16, (3, 3), padding='same', activation='relu')(model)
model =BatchNormalization()(model)
model = Dropout(0.1)(model)

model = Conv2D(10, (4, 4), padding='same', activation='relu')(model)
model =BatchNormalization()(model)
model = Dropout(0.1)(model)




# backbone = ResNet50(weights=None,
#                     include_top=False, 
#                     input_tensor=Input(shape=(224, 224, 3)))
#weights="imagenet",

# neck = backbone.output
neck = model
neck = Flatten(name="flatten")(neck)
neck = Dense(512, activation="relu")(neck)



def build_tower(in_layer):
    neck = Dropout(0.2)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.3)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    return neck


def build_head(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

def build_tower_gender(in_layer):
    neck = Dropout(0.2)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    
    return neck


def build_head_gender(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

def build_tower_image_quality(in_layer):
    neck = Dropout(0.2)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.3)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    return neck


def build_head_image_quality(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

def build_tower_age(in_layer):
    #neck = Dropout(0.2)(in_layer)
    neck = Dense(64, activation="relu")(neck)
    #eck = Dropout(0.3)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.2)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dense(128, activation="relu")(neck)
    return neck


def build_head_age(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

def build_tower_weight(in_layer):
    neck = Dropout(0.1)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.1)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.1)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    return neck


def build_head_weight(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

def build_tower_bag(in_layer):
    #neck = Dropout(0.2)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    # neck = Dropout(0.3)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dense(128, activation="relu")(neck)
    neck = Dense(128, activation="relu")(neck)
    return neck


def build_head_bag(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

def build_tower_footwear(in_layer):
    neck = Dropout(0.1)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.1)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.1)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.1)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    return neck


def build_head_footwear(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

def build_tower_emotion(in_layer):
    neck = Dropout(0.1)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.1)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.1)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.1)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    return neck


def build_head_emotion(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)

def build_tower_pose(in_layer):
    neck = Dropout(0.1)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.1)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.1)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    neck = Dropout(0.1)(in_layer)
    neck = Dense(128, activation="relu")(neck)
    return neck


def build_head_pose(name, in_layer):
    return Dense(
        num_units[name], activation="softmax", name=f"{name}_output"
    )(in_layer)




# heads
gender = build_head_gender("gender", build_tower(neck))
image_quality = build_head_image_quality("image_quality", build_tower(neck))
age = build_head_age("age", build_tower(neck))
weight = build_head_weight("weight", build_tower(neck))
bag = build_head_bag("bag", build_tower(neck))
footwear = build_head_footwear("footwear", build_tower(neck))
emotion = build_head_emotion("emotion", build_tower(neck))
pose = build_head_pose("pose", build_tower(neck))



model = Model(
    #inputs=backbone.input, 
    input = input_img,

    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:203: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=Tensor("in...)`


In [0]:
#cyclic learning rate
from keras.callbacks import *

class CyclicLR(Callback):
    """This callback implements a cyclical learning rate policy (CLR).
    The method cycles the learning rate between two boundaries with
    some constant frequency, as detailed in this paper (https://arxiv.org/abs/1506.01186).
    The amplitude of the cycle can be scaled on a per-iteration or 
    per-cycle basis.
    This class has three built-in policies, as put forth in the paper.
    "triangular":
        A basic triangular cycle w/ no amplitude scaling.
    "triangular2":
        A basic triangular cycle that scales initial amplitude by half each cycle.
    "exp_range":
        A cycle that scales initial amplitude by gamma**(cycle iterations) at each 
        cycle iteration.
    For more detail, please see paper.
    
    # Example
        ```python
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., mode='triangular')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```
    
    Class also supports custom scaling functions:
        ```python
            clr_fn = lambda x: 0.5*(1+np.sin(x*np.pi/2.))
            clr = CyclicLR(base_lr=0.001, max_lr=0.006,
                                step_size=2000., scale_fn=clr_fn,
                                scale_mode='cycle')
            model.fit(X_train, Y_train, callbacks=[clr])
        ```    
    # Arguments
        base_lr: initial learning rate which is the
            lower boundary in the cycle.
        max_lr: upper boundary in the cycle. Functionally,
            it defines the cycle amplitude (max_lr - base_lr).
            The lr at any cycle is the sum of base_lr
            and some scaling of the amplitude; therefore 
            max_lr may not actually be reached depending on
            scaling function.
        step_size: number of training iterations per
            half cycle. Authors suggest setting step_size
            2-8 x training iterations in epoch.
        mode: one of {triangular, triangular2, exp_range}.
            Default 'triangular'.
            Values correspond to policies detailed above.
            If scale_fn is not None, this argument is ignored.
        gamma: constant in 'exp_range' scaling function:
            gamma**(cycle iterations)
        scale_fn: Custom scaling policy defined by a single
            argument lambda function, where 
            0 <= scale_fn(x) <= 1 for all x >= 0.
            mode paramater is ignored 
        scale_mode: {'cycle', 'iterations'}.
            Defines whether scale_fn is evaluated on 
            cycle number or cycle iterations (training
            iterations since start of cycle). Default is 'cycle'.
    """

    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())


In [0]:
# # freeze backbone
# for layer in backbone.layers:
# 	layer.trainable = False

In [0]:
# losses = {
# 	"gender_output": "binary_crossentropy",
# 	"image_quality_output": "categorical_crossentropy",
# 	"age_output": "categorical_crossentropy",
# 	"weight_output": "categorical_crossentropy",

# }
# loss_weights = {"gender_output": 1.0, "image_quality_output": 1.0, "age_output": 1.0}

losses = {
 	"gender_output": "binary_crossentropy",
 	"image_quality_output": "categorical_crossentropy",
 	"age_output": "categorical_crossentropy",
 	"weight_output": "categorical_crossentropy",
  "bag_output": "categorical_crossentropy",
  "footwear_output": "categorical_crossentropy",
  "pose_output": "categorical_crossentropy",
  "emotion_output": "categorical_crossentropy",
 }

loss_weights = {
 	"gender_output": 1.0,
 	"image_quality_output": 1.0,
 	"age_output": 1.0,
 	"weight_output": 1.0,
  "bag_output": 1.0,
  "footwear_output": 1.0,
  "pose_output": 1.0,
  "emotion_output": 1.0,
 }
#opt = SGD(lr=0.001, momentum=0.9)
opt = SGD()
model.compile(
    optimizer=opt,
    loss=losses, 
    loss_weights=loss_weights, 
    metrics=["accuracy"]
)

In [0]:
# model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=32, epochs=10)
filepath="weights-improvement-{epoch:02d}-{loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
clr = CyclicLR(base_lr=0.009, max_lr=0.09,step_size=2000., mode='triangular')
callbacks_list = [checkpoint]


In [0]:
model.save_weights("3mpara_47.h5")

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    # use_multiprocessing=True,
    # workers=6, 
    epochs=5,
    verbose=1,
    callbacks=callbacks_list
)

Epoch 1/5
360/360 [==============================] - 171s 476ms/step - loss: 8.4615 - gender_output_loss: 0.7119 - image_quality_output_loss: 1.0589 - age_output_loss: 1.5354 - weight_output_loss: 1.0807 - bag_output_loss: 0.9994 - footwear_output_loss: 1.0495 - pose_output_loss: 1.0203 - emotion_output_loss: 1.0054 - gender_output_acc: 0.5625 - image_quality_output_acc: 0.5210 - age_output_acc: 0.3602 - weight_output_acc: 0.6155 - bag_output_acc: 0.5294 - footwear_output_acc: 0.5123 - pose_output_acc: 0.6013 - emotion_output_acc: 0.6992 - val_loss: 8.0822 - val_gender_output_loss: 0.7086 - val_image_quality_output_loss: 0.9842 - val_age_output_loss: 1.4624 - val_weight_output_loss: 0.9964 - val_bag_output_loss: 0.9829 - val_footwear_output_loss: 1.0248 - val_pose_output_loss: 0.9653 - val_emotion_output_loss: 0.9575 - val_gender_output_acc: 0.5590 - val_image_quality_output_acc: 0.5444 - val_age_output_acc: 0.3604 - val_weight_output_acc: 0.6396 - val_bag_output_acc: 0.5444 - val_foot

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    # use_multiprocessing=True,
    # workers=1, 
    epochs=20,
    verbose=1
)

Epoch 1/20
360/360 [==============================] - 168s 467ms/step - loss: 7.6058 - gender_output_loss: 0.6249 - image_quality_output_loss: 0.9794 - age_output_loss: 1.4118 - weight_output_loss: 0.9824 - bag_output_loss: 0.9001 - footwear_output_loss: 0.9002 - pose_output_loss: 0.8913 - emotion_output_loss: 0.9156 - gender_output_acc: 0.6430 - image_quality_output_acc: 0.5521 - age_output_acc: 0.3948 - weight_output_acc: 0.6351 - bag_output_acc: 0.5674 - footwear_output_acc: 0.5833 - pose_output_acc: 0.6182 - emotion_output_acc: 0.7090 - val_loss: 7.8996 - val_gender_output_loss: 0.6558 - val_image_quality_output_loss: 1.0159 - val_age_output_loss: 1.4687 - val_weight_output_loss: 1.0258 - val_bag_output_loss: 0.9471 - val_footwear_output_loss: 1.0066 - val_pose_output_loss: 0.8789 - val_emotion_output_loss: 0.9007 - val_gender_output_acc: 0.5907 - val_image_quality_output_acc: 0.5655 - val_age_output_acc: 0.3700 - val_weight_output_acc: 0.6396 - val_bag_output_acc: 0.5449 - val_foo

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    # use_multiprocessing=True,
    # workers=6, 
    epochs=20,
    verbose=1,
    callbacks=callbacks_list
)

Epoch 1/20
360/360 [==============================] - 168s 467ms/step - loss: 7.0201 - gender_output_loss: 0.4840 - image_quality_output_loss: 0.9277 - age_output_loss: 1.3785 - weight_output_loss: 0.9556 - bag_output_loss: 0.8554 - footwear_output_loss: 0.8299 - pose_output_loss: 0.7063 - emotion_output_loss: 0.8827 - gender_output_acc: 0.7645 - image_quality_output_acc: 0.5645 - age_output_acc: 0.4027 - weight_output_acc: 0.6341 - bag_output_acc: 0.6053 - footwear_output_acc: 0.6227 - pose_output_acc: 0.6924 - emotion_output_acc: 0.7092 - val_loss: 7.3609 - val_gender_output_loss: 0.6336 - val_image_quality_output_loss: 0.9453 - val_age_output_loss: 1.4279 - val_weight_output_loss: 0.9557 - val_bag_output_loss: 1.0015 - val_footwear_output_loss: 0.8666 - val_pose_output_loss: 0.6922 - val_emotion_output_loss: 0.8382 - val_gender_output_acc: 0.6966 - val_image_quality_output_acc: 0.5570 - val_age_output_acc: 0.3831 - val_weight_output_acc: 0.6391 - val_bag_output_acc: 0.5489 - val_foo

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    # use_multiprocessing=True,
    # workers=6, 
    epochs=20,
    verbose=1,
    callbacks=callbacks_list
)

Epoch 1/20
360/360 [==============================] - 168s 465ms/step - loss: 6.7487 - gender_output_loss: 0.4209 - image_quality_output_loss: 0.9102 - age_output_loss: 1.3542 - weight_output_loss: 0.9414 - bag_output_loss: 0.8377 - footwear_output_loss: 0.8026 - pose_output_loss: 0.6118 - emotion_output_loss: 0.8699 - gender_output_acc: 0.8021 - image_quality_output_acc: 0.5708 - age_output_acc: 0.4169 - weight_output_acc: 0.6373 - bag_output_acc: 0.6213 - footwear_output_acc: 0.6378 - pose_output_acc: 0.7440 - emotion_output_acc: 0.7089 - val_loss: 6.7066 - val_gender_output_loss: 0.3945 - val_image_quality_output_loss: 0.9188 - val_age_output_loss: 1.3724 - val_weight_output_loss: 0.9446 - val_bag_output_loss: 0.8448 - val_footwear_output_loss: 0.8191 - val_pose_output_loss: 0.5786 - val_emotion_output_loss: 0.8338 - val_gender_output_acc: 0.8261 - val_image_quality_output_acc: 0.5610 - val_age_output_acc: 0.4017 - val_weight_output_acc: 0.6406 - val_bag_output_acc: 0.6200 - val_foo

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    # use_multiprocessing=True,
    # workers=6, 
    epochs=20,
    verbose=1,
    callbacks=callbacks_list
)

Epoch 1/20
360/360 [==============================] - 169s 469ms/step - loss: 6.5612 - gender_output_loss: 0.3843 - image_quality_output_loss: 0.8930 - age_output_loss: 1.3388 - weight_output_loss: 0.9293 - bag_output_loss: 0.8194 - footwear_output_loss: 0.7757 - pose_output_loss: 0.5597 - emotion_output_loss: 0.8610 - gender_output_acc: 0.8267 - image_quality_output_acc: 0.5802 - age_output_acc: 0.4190 - weight_output_acc: 0.6408 - bag_output_acc: 0.6356 - footwear_output_acc: 0.6510 - pose_output_acc: 0.7644 - emotion_output_acc: 0.7094 - val_loss: 6.6648 - val_gender_output_loss: 0.3774 - val_image_quality_output_loss: 0.8980 - val_age_output_loss: 1.3727 - val_weight_output_loss: 0.9413 - val_bag_output_loss: 0.8609 - val_footwear_output_loss: 0.8100 - val_pose_output_loss: 0.5698 - val_emotion_output_loss: 0.8347 - val_gender_output_acc: 0.8317 - val_image_quality_output_acc: 0.5786 - val_age_output_acc: 0.4047 - val_weight_output_acc: 0.6416 - val_bag_output_acc: 0.6084 - val_foo

In [0]:
!ls

gdrive		     weights-improvement-09-6.49.hdf5
hvc_annotations.csv  weights-improvement-09-6.66.hdf5
resized		     weights-improvement-09-6.89.hdf5
sample_data	     weights-improvement-10-6.64.hdf5


In [0]:
# model.fit(X_train, y_train, validation_data=(X_valid, y_valid), batch_size=32, epochs=10)
filepath="weights-improvement.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
clr = CyclicLR(base_lr=0.009, max_lr=0.09,step_size=2000., mode='triangular')
callbacks_list = [checkpoint]


In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    # use_multiprocessing=True,
    # workers=6, 
    epochs=20,
    verbose=1,
    callbacks=callbacks_list
)

Epoch 1/20
360/360 [==============================] - 167s 464ms/step - loss: 6.4289 - gender_output_loss: 0.3628 - image_quality_output_loss: 0.8816 - age_output_loss: 1.3165 - weight_output_loss: 0.9202 - bag_output_loss: 0.8043 - footwear_output_loss: 0.7647 - pose_output_loss: 0.5321 - emotion_output_loss: 0.8468 - gender_output_acc: 0.8367 - image_quality_output_acc: 0.5830 - age_output_acc: 0.4280 - weight_output_acc: 0.6438 - bag_output_acc: 0.6414 - footwear_output_acc: 0.6576 - pose_output_acc: 0.7805 - emotion_output_acc: 0.7105 - val_loss: 6.9406 - val_gender_output_loss: 0.5434 - val_image_quality_output_loss: 0.9038 - val_age_output_loss: 1.3849 - val_weight_output_loss: 0.9494 - val_bag_output_loss: 0.8567 - val_footwear_output_loss: 0.8351 - val_pose_output_loss: 0.6244 - val_emotion_output_loss: 0.8429 - val_gender_output_acc: 0.7581 - val_image_quality_output_acc: 0.5706 - val_age_output_acc: 0.3997 - val_weight_output_acc: 0.6371 - val_bag_output_acc: 0.6074 - val_foo

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    # use_multiprocessing=True,
    # workers=6, 
    epochs=20,
    verbose=1,
    callbacks=callbacks_list
)

Epoch 1/20
360/360 [==============================] - 167s 463ms/step - loss: 6.2851 - gender_output_loss: 0.3386 - image_quality_output_loss: 0.8697 - age_output_loss: 1.2979 - weight_output_loss: 0.9017 - bag_output_loss: 0.7844 - footwear_output_loss: 0.7501 - pose_output_loss: 0.5044 - emotion_output_loss: 0.8382 - gender_output_acc: 0.8479 - image_quality_output_acc: 0.5885 - age_output_acc: 0.4367 - weight_output_acc: 0.6453 - bag_output_acc: 0.6548 - footwear_output_acc: 0.6648 - pose_output_acc: 0.7960 - emotion_output_acc: 0.7113 - val_loss: 6.8252 - val_gender_output_loss: 0.4436 - val_image_quality_output_loss: 0.9111 - val_age_output_loss: 1.3886 - val_weight_output_loss: 0.9445 - val_bag_output_loss: 0.8482 - val_footwear_output_loss: 0.8532 - val_pose_output_loss: 0.5924 - val_emotion_output_loss: 0.8436 - val_gender_output_acc: 0.8080 - val_image_quality_output_acc: 0.5801 - val_age_output_acc: 0.4047 - val_weight_output_acc: 0.6366 - val_bag_output_acc: 0.6200 - val_foo

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    # use_multiprocessing=True,
    # workers=6, 
    epochs=20,
    verbose=1,
    callbacks=callbacks_list
)

Epoch 1/20
360/360 [==============================] - 170s 473ms/step - loss: 6.1705 - gender_output_loss: 0.3208 - image_quality_output_loss: 0.8611 - age_output_loss: 1.2741 - weight_output_loss: 0.8853 - bag_output_loss: 0.7788 - footwear_output_loss: 0.7372 - pose_output_loss: 0.4849 - emotion_output_loss: 0.8284 - gender_output_acc: 0.8622 - image_quality_output_acc: 0.5905 - age_output_acc: 0.4462 - weight_output_acc: 0.6483 - bag_output_acc: 0.6580 - footwear_output_acc: 0.6737 - pose_output_acc: 0.8009 - emotion_output_acc: 0.7126 - val_loss: 6.9607 - val_gender_output_loss: 0.3643 - val_image_quality_output_loss: 1.0540 - val_age_output_loss: 1.4480 - val_weight_output_loss: 0.9713 - val_bag_output_loss: 0.8589 - val_footwear_output_loss: 0.8337 - val_pose_output_loss: 0.5809 - val_emotion_output_loss: 0.8495 - val_gender_output_acc: 0.8427 - val_image_quality_output_acc: 0.5156 - val_age_output_acc: 0.3987 - val_weight_output_acc: 0.6431 - val_bag_output_acc: 0.6200 - val_foo

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    # use_multiprocessing=True,
    # workers=6, 
    epochs=20,
    verbose=1,
    callbacks=callbacks_list
)

Epoch 1/20
360/360 [==============================] - 169s 470ms/step - loss: 6.0683 - gender_output_loss: 0.3154 - image_quality_output_loss: 0.8556 - age_output_loss: 1.2585 - weight_output_loss: 0.8783 - bag_output_loss: 0.7648 - footwear_output_loss: 0.7235 - pose_output_loss: 0.4571 - emotion_output_loss: 0.8150 - gender_output_acc: 0.8640 - image_quality_output_acc: 0.5987 - age_output_acc: 0.4577 - weight_output_acc: 0.6531 - bag_output_acc: 0.6627 - footwear_output_acc: 0.6774 - pose_output_acc: 0.8202 - emotion_output_acc: 0.7161 - val_loss: 6.7491 - val_gender_output_loss: 0.3872 - val_image_quality_output_loss: 0.9192 - val_age_output_loss: 1.3957 - val_weight_output_loss: 0.9587 - val_bag_output_loss: 0.8551 - val_footwear_output_loss: 0.8181 - val_pose_output_loss: 0.5579 - val_emotion_output_loss: 0.8574 - val_gender_output_acc: 0.8387 - val_image_quality_output_acc: 0.5670 - val_age_output_acc: 0.4047 - val_weight_output_acc: 0.6341 - val_bag_output_acc: 0.6195 - val_foo

In [0]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    # use_multiprocessing=True,
    # workers=6, 
    epochs=20,
    verbose=1,
    callbacks=callbacks_list
)

Epoch 1/20
360/360 [==============================] - 169s 470ms/step - loss: 5.9750 - gender_output_loss: 0.3071 - image_quality_output_loss: 0.8389 - age_output_loss: 1.2389 - weight_output_loss: 0.8596 - bag_output_loss: 0.7479 - footwear_output_loss: 0.7186 - pose_output_loss: 0.4557 - emotion_output_loss: 0.8083 - gender_output_acc: 0.8676 - image_quality_output_acc: 0.6087 - age_output_acc: 0.4613 - weight_output_acc: 0.6582 - bag_output_acc: 0.6754 - footwear_output_acc: 0.6797 - pose_output_acc: 0.8144 - emotion_output_acc: 0.7163 - val_loss: 6.7169 - val_gender_output_loss: 0.3477 - val_image_quality_output_loss: 0.9017 - val_age_output_loss: 1.3950 - val_weight_output_loss: 0.9613 - val_bag_output_loss: 0.8563 - val_footwear_output_loss: 0.8070 - val_pose_output_loss: 0.5851 - val_emotion_output_loss: 0.8629 - val_gender_output_acc: 0.8508 - val_image_quality_output_acc: 0.5736 - val_age_output_acc: 0.3997 - val_weight_output_acc: 0.6381 - val_bag_output_acc: 0.6169 - val_foo